# 1. 💾 Arrays vs. Python Lists (The Hardware Reality)

**Key Topics Covered:**
* **Hardware Reality:** Contiguous Memory & Pointer Arithmetic.
* **The "Dynamic" Illusion:** How Python Lists resize.
* **Types of Arrays:** `list` vs. `array` vs. `numpy`.
* **Performance Trap:** Why `insert(0)` destroys performance.

## 1.1 🧠 The Hardware Reality: Contiguous Memory

Why is accessing `my_array[5]` instant ($O(1)$)? It's not magic; it's math. 

An array reserves a solid block of RAM. If the array starts at memory address `1000` and holds Integers (4 bytes each), the computer calculates the address of index `5` using simple arithmetic:

$$ \text{Address} = \text{Start} + (\text{Index} \times \text{Element\_Size}) $$
$$ \text{Address} = 1000 + (5 \times 4) = 1020 $$

The CPU jumps directly to address `1020`. It doesn't search. This is why Arrays are the king of speed. (less time)

But in search Arrays, it's not always the best choice. It is not searching for 'index' but for 'value'. Computer have no idea about a `value` so it has to search for it from the beginning. so much time.

## 1.2 🪄 Python Lists: The "Dynamic" Array

Traditional arrays (C/Java) are **Fixed Size**. Python Lists are **Dynamic Arrays**. Also **Heterogeneous**. But a array must be *homogeneous*.

**How it works:**
1.  Python allocates a small block (e.g., 4 slots).
2.  When you fill it, Python doesn't just add one slot. It allocates a **NEW, BIGGER block** (usually 1.125x or 2x larger).
3.  It copies all old items to the new block.
4.  It deletes the old block.

This strategy is called **Amortization**. Most appends are cheap ($O(1)$), but occasionally one is expensive ($O(N)$).

In [1]:
import sys

# Let's watch a list grow!
my_list = []
current_capacity = sys.getsizeof(my_list)

print(f"Initial Capacity: {current_capacity} bytes")

for i in range(50):
    my_list.append(i)
    new_capacity = sys.getsizeof(my_list)
    
    # Only print when the capacity CHANGES (the resize event)
    if new_capacity > current_capacity:
        print(f"⚠️ Resized! Length: {len(my_list):2d} | New Capacity: {new_capacity} bytes")
        current_capacity = new_capacity

Initial Capacity: 56 bytes
⚠️ Resized! Length:  1 | New Capacity: 88 bytes
⚠️ Resized! Length:  5 | New Capacity: 120 bytes
⚠️ Resized! Length:  9 | New Capacity: 184 bytes
⚠️ Resized! Length: 17 | New Capacity: 248 bytes
⚠️ Resized! Length: 25 | New Capacity: 312 bytes
⚠️ Resized! Length: 33 | New Capacity: 376 bytes
⚠️ Resized! Length: 41 | New Capacity: 472 bytes


## 1.3 📦 Types of Arrays in Python

As a CSE student, you must distinguish between these three:

1. **Python List:** Flexible types, heavy memory.
2. **array.array:** Fixed types, light memory.
3. **numpy.array:** Fixed types, light memory.

In [ ]:
import sys
import array # Standard library
import numpy as np # Data Science library: pip install numpy

# 1. Python List (References)
# Stores POINTERS to objects. Flexible types, heavy memory.
py_list = [1, 2, 3]

# 2. 'array' Module (Values)
# Stores raw C values (bytes). Fixed type (e.g., 'i' for signed int), lower memory.
c_array = array.array('i', [1, 2, 3])

# 3. NumPy Array (The Data Science King)
# Contiguous C array + Powerful Math Processor.
np_array = np.array([1, 2, 3])

print(f"List: {py_list} | Size: {sys.getsizeof(py_list)} bytes")
print(f"Array: {c_array} | Size: {sys.getsizeof(c_array)} bytes")
print(f"NumPy: {np_array} | Size: {sys.getsizeof(np_array)} bytes")

List: [1, 2, 3] | Size: 88 bytes
Array: array('i', [1, 2, 3]) | Size: 92 bytes
NumPy: [1 2 3] | Size: 136 bytes


### More about `array` module:

| Type Code | C Type | Python Type | Minimum size in bytes |
|---------|--------|-------------|-----------------------|
| 'b' | signed char | int | 1 |
| 'B' | unsigned char | int | 1 |
| 'u' | Py_UNICODE | Unicode character | 2 |
| 'h' | signed short | int | 2 |
| 'H' | unsigned short | int | 2 |
| 'i' | signed int | int | 2 |
| 'I' | unsigned int | int | 2 |
| 'l' | signed long | int | 4 |
| 'L' | unsigned long | int | 4 |
| 'q' | signed long long | int | 8 |
| 'Q' | unsigned long long | int | 8 |
| 'f' | float | float | 4 |
| 'd' | double | float | 8 |

* Unsinged means it can only store positive values.
* Signed means it can store both positive and negative values.

---

## ❔ Mini-Challenge: The Performance Trap

Let's prove Big-O notation is real. We will build a list of 50,000 items in two ways:
1.  **Append:** Adding to the end ($O(1)$).
2.  **Insert:** Adding to the start ($O(N)$).

**Your Task:** Run the code and observe the time difference.

In [4]:
import time

N = 50000 # Number of items

# --- Test 1: Append (End) ---
start = time.time()
list_a = []
for i in range(N):
    list_a.append(i)
end = time.time()
print(f"Append Time: {end - start:.5f} seconds")

# --- Test 2: Insert (Start) ---
start = time.time()
list_b = []
for i in range(N):
    list_b.insert(0, i) # This forces a shift of ALL items every loop!
end = time.time()
print(f"Insert Time: {end - start:.5f} seconds")

# The Result? Insert should be MASSIVELY slower.

Append Time: 0.00201 seconds
Insert Time: 0.27723 seconds


> Best way is append the elements to the array!

> Delete the element from the array is not a good idea also. It is better to delete the element from the end will reduce the time complexity to O(1).

---

## 1.4 🌍 Real-World System Map

Where are Arrays actually used?

### 1. Image Processing (The Matrix)
*   **Example:** **Photoshop Filters / Computer Vision**.
*   **Why?** An image is just a massive 2D array of pixels `[Height][Width]`. To apply a "Grayscale" filter, the CPU iterates through this array. Because arrays are contiguous in RAM, the CPU can preload chunks of the image into the Cache (L1/L2), making processing lightning fast. Using a Linked List here would be millions of times slower.

### 2. Time Series Data
*   **Example:** **Stock Market Tickers**.
*   **Why?** Prices are historical data (`time` on X-axis). You rarely insert a new price in the *middle* of history (no `insert` at index 0). You only `append` new prices at the end. Lists are perfect for this `append-only` workload.

## 1.5 Sorted Array

When we have a sorted array, we can use binary search to find an element in O(log n) time. Inserting and deleting elements from a sorted array is also O(log n) time. It is a good idea to keep the array sorted when we need to perform these operations.